In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [9]:
# Update the file path based on the correct location provided by the user
file_path = "/workspaces/Rag_Project_Pod/Data_prep/data.csv"

# Try loading the CSV file again
df = pd.read_csv(file_path)
df.head()


,Category,Question,Answer
0,General Information,What is syndicated research?,Syndicated research is a type of market resear...
1,General Information,How often is the data updated?,The data is typically updated on a monthly bas...
2,General Information,What FMCG categories are covered in this resea...,The research covers a wide range of FMCG categ...
3,General Information,Who can benefit from this syndicated research?,This research is valuable for FMCG manufacture...
4,General Information,What geographic regions are covered in this re...,"The research includes data from North America,..."


In [10]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
documents = df.to_dict(orient='records')
documents[0]

{'Category': 'General Information',
 'Question': 'What is syndicated research?',
 'Answer': 'Syndicated research is a type of market research where data and findings are collected and compiled by a research agency and then sold to multiple clients. It provides insights into consumer behavior, market trends, and product performance across various FMCG categories.'}

In [12]:
for doc in tqdm(documents):
    question = doc['Question']
    text = doc['Answer']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/260 [00:00<?, ?it/s]